In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/12-23대기오염nan처리.csv')

In [3]:
df.head(1)

,측정일시,이산화질소,오존,일산화탄소,아황산,미세,초미세
0,20120101,0.031,0.008,1.17,0.0093,88.3,57.3


In [4]:
df.tail(1)

,측정일시,이산화질소,오존,일산화탄소,아황산,미세,초미세
4303,20231211,0.015,0.0227,0.4,0.0024,7.2,3.0


In [5]:
df['측정일시'] = pd.to_datetime(df['측정일시'], format='%Y%m%d')

In [6]:
df.head(1)

,측정일시,이산화질소,오존,일산화탄소,아황산,미세,초미세
0,2012-01-01,0.031,0.008,1.17,0.0093,88.3,57.3


In [7]:
df = df.set_index('측정일시')

In [8]:
df.index

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08',
               '2012-01-09', '2012-01-10',
               ...
               '2023-12-02', '2023-12-03', '2023-12-04', '2023-12-05',
               '2023-12-06', '2023-12-07', '2023-12-08', '2023-12-09',
               '2023-12-10', '2023-12-11'],
              dtype='datetime64[ns]', name='측정일시', length=4304, freq=None)

In [9]:
df.isna().sum()

이산화질소    0
오존       0
일산화탄소    0
아황산      0
미세       0
초미세      0
dtype: int64

In [10]:
df

,이산화질소,오존,일산화탄소,아황산,미세,초미세
측정일시,,,,,,
2012-01-01,0.031,0.0080,1.17,0.0093,88.3,57.3
2012-01-02,0.036,0.0050,1.20,0.0087,100.0,62.7
2012-01-03,0.041,0.0063,1.27,0.0073,96.0,63.0
2012-01-04,0.024,0.0130,0.83,0.0103,70.3,42.0
2012-01-05,0.037,0.0073,0.77,0.0063,53.3,29.3
...,...,...,...,...,...,...
2023-12-07,0.026,0.0184,0.50,0.0033,62.2,16.5
2023-12-08,0.029,0.0300,0.56,0.0031,84.2,22.8
2023-12-09,0.027,0.0379,0.61,0.0029,72.3,30.7


In [ ]:
df.head(1)

,이산화질소,오존,일산화탄소,아황산,미세,초미세
측정일시,,,,,,
2012-01-01,0.036,0.009,1.0,0.01,83.0,57.0


In [ ]:
df.columns

Index(['Unnamed: 0', '이산화질소', '오존', '일산화탄소', '아황산', '미세', '초미세'], dtype='object')

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator

In [27]:
df.columns

Index(['이산화질소', '오존', '일산화탄소', '아황산', '미세', '초미세'], dtype='object')

In [45]:
from sklearn.preprocessing import MinMaxScaler

# 피쳐 및 타겟 분리
features = df[['이산화질소', '오존', '일산화탄소', '아황산', '초미세']]
target = df['미세']

# 피쳐 스케일링
scaler_features = MinMaxScaler()
features_scaled = scaler_features.fit_transform(features)

# 타겟 스케일링
scaler_target = MinMaxScaler()
target_scaled = scaler_target.fit_transform(target.values.reshape(-1, 1))


In [46]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 30
n_features = features.shape[1]
generator = TimeseriesGenerator(features_scaled, target_scaled, length=n_input, batch_size=1)


In [47]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# EarlyStopping 콜백 설정 조기종료
early_stopping = EarlyStopping(monitor='loss', patience=10)


In [48]:
# 모델 훈련 with EarlyStopping
model.fit(generator, epochs=100, callbacks=[early_stopping])


Epoch 1/100


4274/4274 [==============================] - 30s 7ms/step - loss: 0.0019
Epoch 2/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0016
Epoch 3/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0016
Epoch 4/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0016
Epoch 5/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0016
Epoch 6/100
4274/4274 [==============================] - 30s 7ms/step - loss: 0.0016
Epoch 7/100
4274/4274 [==============================] - 30s 7ms/step - loss: 0.0016
Epoch 8/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0015
Epoch 9/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0015
Epoch 10/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0015
Epoch 11/100
4274/4274 [==============================] - 29s 7ms/step - loss: 0.0015
Epoch 12/100
4274/4274 [==============================] - 29s 7ms/step - lo

In [33]:
df.tail()

,이산화질소,오존,일산화탄소,아황산,미세,초미세
측정일시,,,,,,
2023-12-07,0.026,0.0184,0.50,0.0033,62.2,16.5
2023-12-08,0.029,0.0300,0.56,0.0031,84.2,22.8
2023-12-09,0.027,0.0379,0.61,0.0029,72.3,30.7
2023-12-10,0.025,0.0209,0.67,0.0029,80.2,35.6
2023-12-11,0.015,0.0227,0.40,0.0024,7.2,3.0


In [49]:
from sklearn.metrics import r2_score

# 테스트 데이터셋 준비 (예시)
# 이 부분은 실제 데이터셋과 상황에 맞게 조정해야 합니다.
# test_features_scaled, test_target_scaled는 테스트 데이터의 스케일된 피쳐와 타겟입니다.
test_generator = TimeseriesGenerator(features_scaled , target_scaled, length=n_input, batch_size=1)

# 테스트 데이터셋에 대한 예측 수행
test_predictions = model.predict(test_generator)

# 예측값 역 스케일링
test_predictions_inverse = scaler_target.inverse_transform(test_predictions)

# 실제 타겟 값 역 스케일링
actual_target = scaler_target.inverse_transform(target_scaled[n_input:])

# R^2 값 계산
r2 = r2_score(actual_target, test_predictions_inverse)
print(f'R^2 Score: {r2}')


4274/4274 [==============================] - 11s 2ms/step
R^2 Score: 0.5005735436162553


In [51]:
# 최근 데이터를 사용한 예측
x_input = features_scaled[-n_input:]
x_input = x_input.reshape((1, n_input, n_features))
predicted = model.predict(x_input, verbose=0)

# 예측값 역 스케일링
predicted_inverse = scaler_target.inverse_transform(predicted)
print(predicted_inverse)


[[15.164758]]


In [52]:
model.save('model/a/pm10_model.h5')

In [55]:
from keras.models import load_model

# 저장된 모델 불러오기
model_pm10 = load_model('model/a/pm10_model.h5')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 52: invalid start byte